In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2021-12-12 20:48:02--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 173.194.195.113, 173.194.195.102, 173.194.195.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.195.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/rh8krudu2o3g6avikao1v5rbt4sr7743/1639342050000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2021-12-12 20:48:04--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/rh8krudu2o3g6avikao1v5rbt4sr7743/1639342050000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 142.250.125.132, 2607:f8b0:4001:c2f::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [ ]:
# импорт пакетов
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [ ]:
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,...,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,OwnerOccupier,Nagornoe,5.293465e+06,77878,0.023464,0.195781,4713,2279.0,4,5212,10027.0,8,1,NaN,3,0,2,2,no,0,2,6,no,no,no,no,no,no,no,...,705.32,1186.17,945.74,6,26,32,25,10,1,0,4,12,1,4,31,8,15.19,19.83,92,3478168,49,2235425,348,755.99,1263.47,1009.73,14,85,113,91,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Poselenie Sosenskoe,6.677245e+07,9553,0.336177,0.072158,656,NaN,0,629,NaN,0,0,NaN,0,0,1,0,no,0,0,1,no,no,no,no,no,yes,no,...,1750.00,2875.00,2312.50,0,0,1,0,1,2,0,0,4,0,0,1,0,34.01,8.70,1,85159,1,189076,14,1108.33,1833.33,1470.83,2,1,4,3,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,3.0,Investment,Krylatskoe,1.216448e+07,78507,0.297166,0.000000,4611,3092.0,7,5067,7478.0,7,0,NaN,2,0,14,1,no,0,6,4,no,no,no,no,no,no,no,...,987.50,1645.83,1316.67,7,8,18,9,7,5,1,2,8,0,0,17,0,35.23,4.58,19,755824,19,1015107,95,923.26,1523.26,1223.26,9,18,25,23,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Matushkino,4.708040e+06,38075,0.271702,0.310199,2448,2080.0,3,2748,3885.0,4,0,NaN,0,0,0,0,no,0,0,1,no,no,no,no,no,no,no,...,500.00,850.00,675.00,0,5,3,2,0,0,0,1,2,0,1,4,1,20.84,7.28,3,51038,7,49700,14,507.14,857.14,682.14,0,7,4,3,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,2.0,Investment,Gol'janovo,1.428699e+07,157010,0.389354,0.194489,7751,5041.0,6,8004,11081.0,7,0,125.0,3,0,5,3,no,0,5,3,no,no,no,yes,no,no,no,...,608.33,1020.83,814.58,2,7,9,8,0,0,0,3,3,0,0,13,2,36.40,9.08,11,300476,16,509176,54,656.00,1110.00,883.00,4,16,18,12,3,1,0,5,8,0,1,34,5,7900000


In [ ]:
df = df.drop('id',1)

In [ ]:
df.select_dtypes(include=['object']).isnull().sum().sum()

0

In [ ]:
numeric = df.select_dtypes(include=[np.number])
numeric_cols = numeric.columns.values
 
for col in numeric_cols:
    missing = df[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        df['{}_ismissing'.format(col)] = missing
        med = df[col].median()
        df[col] = df[col].fillna(med)

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [ ]:
from sklearn.preprocessing import LabelEncoder
def load_dataset(data):
    X, y = data.drop('price',1), data.price

    num_index = X.select_dtypes(include=[np.number]).keys()

    objects = X.select_dtypes(include=['object']).keys()


    lbl = LabelEncoder()

    for col in objects:

        lbl.fit(list(X[col].values)) 
        
        X[col] = lbl.transform(list(X[col].values))
    
    return X, y, num_index

In [ ]:
X, y, num_index = load_dataset(df.drop('timestamp',1))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[num_index] = scaler.fit_transform(X_train[num_index])

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_model = DecisionTreeRegressor()
tree_model = tree_model.fit(X_train,y_train)

Проверьте качество на отложенной выборке.

In [ ]:
from sklearn.metrics import mean_squared_error
X_test[num_index] = scaler.transform(X_test[num_index])
np.sqrt(mean_squared_error(y_test,tree_model.predict(X_test)))

3761435.302704087

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [ ]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)

month_year_cnt_map = month_year.value_counts().to_dict()

df["month_year_cnt"] = month_year.map(month_year_cnt_map)



week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)

week_year_cnt_map = week_year.value_counts().to_dict()

df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [ ]:
df['Month'] = df['timestamp'].map(lambda x: x.month)
df['Weekday'] = df['timestamp'].map(lambda x: x.isoweekday())
df['floor/max_floor'] = df['floor'] / df['max_floor']
df['kitch_sq/full_sq'] = df['kitch_sq'] / df['full_sq']
df.replace([np.inf], 1, inplace=True)
df = df.fillna(1)

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [ ]:
X, y, num_index = load_dataset(df.drop('timestamp',1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
scaler = StandardScaler()
X_train[num_index] = scaler.fit_transform(X_train[num_index])

In [ ]:
tree_model = DecisionTreeRegressor()
tree_model = tree_model.fit(X_train,y_train)

In [ ]:
X_test[num_index] = scaler.transform(X_test[num_index])
np.sqrt(mean_squared_error(y_test,tree_model.predict(X_test)))

3920645.25131966

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.3 MB/s 


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

1.Решающие деревья

In [ ]:
param_grid = {"max_depth" : [1,3,9,11],
           "min_samples_leaf":[1,5,9],
           "max_features":["auto","sqrt"]}
DTRgrid_search = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param_grid, n_jobs=-1,
                              scoring='neg_mean_squared_error')
DTRgrid_result = DTRgrid_search.fit(X_train, y_train)
print(DTRgrid_result.best_params_)

{'max_depth': 9, 'max_features': 'auto', 'min_samples_leaf': 9}


In [ ]:
tree_model_best= DTRgrid_result.best_estimator_

tree_model_best.fit(X_train,y_train)

np.sqrt(mean_squared_error(y_test,tree_model_best.predict(X_test)))

3291911.4518001904

2. Cлучайный лес

In [ ]:
param_grid = {'n_estimators': [25,215,300],
              'max_depth': [2, 3], 
              'min_samples_split': [1, 2, 3]}


RFRgrid_search = GridSearchCV(RandomForestRegressor(), param_grid=param_grid, n_jobs=-1,
                              scoring='neg_root_mean_squared_error')


RFRgrid_result = RFRgrid_search.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 459, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/para

In [ ]:
rf_model_best= RFRgrid_result.best_estimator_
rf_model_best.fit(X_train,y_train)
np.sqrt(mean_squared_error(y_test,rf_model_best.predict(X_test)))

3241212.572612183

3. CatBoost(градиентный бустинг)

In [ ]:
param_grid = {'depth' : [4,6,8],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [10, 50, 100]
              }

CBRgrid_search = GridSearchCV(CatBoostRegressor(), param_grid=param_grid, n_jobs=-1,
                              scoring='neg_root_mean_squared_error')
CBRgrid_result = CBRgrid_search.fit(X_train, y_train)

0:	learn: 4380206.8657556	total: 124ms	remaining: 12.2s
1:	learn: 4181274.8740876	total: 179ms	remaining: 8.78s
2:	learn: 3999908.3883089	total: 240ms	remaining: 7.76s
3:	learn: 3830429.4484910	total: 299ms	remaining: 7.18s
4:	learn: 3684937.6625245	total: 365ms	remaining: 6.94s
5:	learn: 3556220.2413877	total: 426ms	remaining: 6.67s
6:	learn: 3446945.1532014	total: 492ms	remaining: 6.53s
7:	learn: 3348500.9788306	total: 549ms	remaining: 6.31s
8:	learn: 3259990.5200659	total: 624ms	remaining: 6.31s
9:	learn: 3183712.0888434	total: 683ms	remaining: 6.14s
10:	learn: 3122499.6882390	total: 739ms	remaining: 5.98s
11:	learn: 3064568.2192799	total: 797ms	remaining: 5.84s
12:	learn: 3016389.2804521	total: 860ms	remaining: 5.75s
13:	learn: 2964128.4511097	total: 926ms	remaining: 5.69s
14:	learn: 2924127.6865273	total: 991ms	remaining: 5.61s
15:	learn: 2889596.0333987	total: 1.05s	remaining: 5.53s
16:	learn: 2858835.2295748	total: 1.11s	remaining: 5.44s
17:	learn: 2825973.8974312	total: 1.17s	r

In [ ]:
CBR_model_best= CBRgrid_result.best_estimator_
CBR_model_best.fit(X_train,y_train)
np.sqrt(mean_squared_error(y_test,CBR_model_best.predict(X_test)))

0:	learn: 4380206.8657556	total: 76.1ms	remaining: 7.54s
1:	learn: 4181274.8740876	total: 144ms	remaining: 7.07s
2:	learn: 3999908.3883089	total: 212ms	remaining: 6.85s
3:	learn: 3830429.4484910	total: 297ms	remaining: 7.13s
4:	learn: 3684937.6625245	total: 360ms	remaining: 6.83s
5:	learn: 3556220.2413877	total: 418ms	remaining: 6.54s
6:	learn: 3446945.1532014	total: 482ms	remaining: 6.4s
7:	learn: 3348500.9788306	total: 549ms	remaining: 6.31s
8:	learn: 3259990.5200659	total: 615ms	remaining: 6.21s
9:	learn: 3183712.0888434	total: 679ms	remaining: 6.11s
10:	learn: 3122499.6882390	total: 744ms	remaining: 6.02s
11:	learn: 3064568.2192799	total: 808ms	remaining: 5.93s
12:	learn: 3016389.2804521	total: 869ms	remaining: 5.81s
13:	learn: 2964128.4511097	total: 937ms	remaining: 5.76s
14:	learn: 2924127.6865273	total: 1000ms	remaining: 5.66s
15:	learn: 2889596.0333987	total: 1.06s	remaining: 5.56s
16:	learn: 2858835.2295748	total: 1.12s	remaining: 5.46s
17:	learn: 2825973.8974312	total: 1.18s	

2795602.434957747

Исходя из результатов можно прийти к выводу что градиентный бустинг лучше в данной ситуации


### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [ ]:
invest_train_X = X_train[X_train['product_type'] == 0]
invest_train_Y = y_train[X_train['product_type'] == 0]
owner_train_X = X_train[X_train['product_type'] == 1]
owner_train_Y = y_train[X_train['product_type'] == 1]

1.Деревья

In [ ]:
model_invest= DTRgrid_result.best_estimator_
model_invest.fit(invest_train_X,invest_train_Y)
np.sqrt(mean_squared_error(y_test,model_invest.predict(X_test)))

3519205.95660484

In [ ]:
model_owner= DTRgrid_result.best_estimator_
model_owner.fit(owner_train_X,owner_train_Y)
np.sqrt(mean_squared_error(y_test,model_owner.predict(X_test)))

3938674.502219069

2. Случайный лес

In [ ]:
model_invest= RFRgrid_result.best_estimator_
model_invest.fit(invest_train_X,invest_train_Y)
np.sqrt(mean_squared_error(y_test,model_invest.predict(X_test)))

3496365.3169175065

In [ ]:
model_owner= RFRgrid_result.best_estimator_
model_owner.fit(owner_train_X,owner_train_Y)
np.sqrt(mean_squared_error(y_test,model_owner.predict(X_test)))

3425625.885746399

3. Градиентный бустинг

In [ ]:
model_invest= CBRgrid_result.best_estimator_
model_invest.fit(invest_train_X,invest_train_Y)
np.sqrt(mean_squared_error(y_test,model_invest.predict(X_test)))

0:	learn: 4592843.6926204	total: 64ms	remaining: 6.33s
1:	learn: 4389383.1133418	total: 117ms	remaining: 5.71s
2:	learn: 4216110.8322277	total: 173ms	remaining: 5.58s
3:	learn: 4070230.7733148	total: 224ms	remaining: 5.38s
4:	learn: 3932771.0413351	total: 292ms	remaining: 5.55s
5:	learn: 3809054.4479512	total: 351ms	remaining: 5.5s
6:	learn: 3704032.9550633	total: 410ms	remaining: 5.44s
7:	learn: 3608635.3034950	total: 476ms	remaining: 5.47s
8:	learn: 3525940.3663508	total: 532ms	remaining: 5.38s
9:	learn: 3447524.3540554	total: 585ms	remaining: 5.27s
10:	learn: 3390397.5915226	total: 643ms	remaining: 5.21s
11:	learn: 3334265.6588963	total: 698ms	remaining: 5.12s
12:	learn: 3277597.4407007	total: 764ms	remaining: 5.12s
13:	learn: 3222971.5053091	total: 814ms	remaining: 5s
14:	learn: 3183557.8262544	total: 865ms	remaining: 4.9s
15:	learn: 3146993.2989674	total: 920ms	remaining: 4.83s
16:	learn: 3111287.1547121	total: 978ms	remaining: 4.77s
17:	learn: 3080631.7374124	total: 1.03s	remaini

3019425.2325807516

In [ ]:
model_owner= CBRgrid_result.best_estimator_
model_owner.fit(owner_train_X,owner_train_Y)
np.sqrt(mean_squared_error(y_test,model_owner.predict(X_test)))

0:	learn: 3858286.8657026	total: 44.8ms	remaining: 4.43s
1:	learn: 3613686.8677799	total: 82ms	remaining: 4.02s
2:	learn: 3392881.4536722	total: 118ms	remaining: 3.82s
3:	learn: 3206492.1652121	total: 154ms	remaining: 3.7s
4:	learn: 3005628.5826547	total: 189ms	remaining: 3.6s
5:	learn: 2851673.8726740	total: 232ms	remaining: 3.63s
6:	learn: 2708185.7598191	total: 278ms	remaining: 3.7s
7:	learn: 2566264.8365445	total: 321ms	remaining: 3.7s
8:	learn: 2434879.6738122	total: 367ms	remaining: 3.71s
9:	learn: 2331081.0495089	total: 410ms	remaining: 3.69s
10:	learn: 2231504.3379635	total: 450ms	remaining: 3.64s
11:	learn: 2137066.5954019	total: 498ms	remaining: 3.65s
12:	learn: 2067312.3477352	total: 536ms	remaining: 3.58s
13:	learn: 1997032.0492744	total: 574ms	remaining: 3.53s
14:	learn: 1928729.8892213	total: 612ms	remaining: 3.46s
15:	learn: 1872878.4647260	total: 650ms	remaining: 3.41s
16:	learn: 1824885.9120722	total: 688ms	remaining: 3.36s
17:	learn: 1784767.3891114	total: 733ms	remai

3227451.0354067907

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [ ]:
from sklearn.linear_model import HuberRegressor

In [ ]:
w1 = 0.9
w2 = 1 - w1

In [ ]:
huber_model= HuberRegressor()
huber_model.fit(invest_train_X, invest_train_Y)
np.sqrt(mean_squared_error(y_test,model_invest.predict(X_test)*w1 + huber_model.predict(X_test)*w2))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


3189966.22232641

Результат улучшен, по сравнению с ансамблем, но проигрывает кэтбусту
